In [3]:
import sys
sys.path.append('../../..')
from Library.lossSRM import LossAbs
from Library.CompoundPoisson2 import CompoundPoisson2
import numpy as np
import scipy.stats as st
from scipy.optimize import minimize

In [4]:
#class for loss function of the following form :  l(x) = 1 / (1 + alpha)(1/b1 * sum(exp(b1xi)) + alpha*exp(b2*sum(xi)))- 1
class MCLossFunction2(LossAbs):
    def __init__(self, X, alpha, b1 = 1, b2 = 1, c = None):
        self.__alpha = alpha
        self.__b1 = b1
        self.__b2 = b2
        super(MCLossFunction2, self).__init__(X, c)
        
    def shortfall_risk(self, m = None):
        m = self._check_argument(m)
        b1, b2, alpha = self.__b1, self.__b2, self.__alpha
        #This substract from the col i of X, m_i
        #X is a matrix of dim columns and N rows where N is the sample's length
        x_minus_m = np.subtract(self.X, m)
        exp_x_minus_m = (1 / b1) * np.exp(b1 * x_minus_m)
        #axis = 1 means we sum the elements of each row
        mean_sum1 = np.mean(np.sum(exp_x_minus_m, axis = 1))
        if alpha == 0:
            mean_sum2 = 0
        else:
            mean_sum2 = alpha * np.mean(np.exp(b2 * np.sum(x_minus_m, axis = 1)))
        return (1 / (1 + alpha)) * (mean_sum1 + mean_sum2) - 1
    
    def shortfall_risk_jac(self, m):
        m = self._check_argument(m)
        x_minus_m = np.subtract(self.X, m)
        alpha = self.__alpha
        if alpha == 0:
            common_mean = 0
        else:
            common_mean = alpha * b2 * np.mean(np.exp(b2 * np.sum(x_minus_m, axis = 1)))
        ind_mean = np.mean(np.exp(b1 * x_minus_m), axis = 0)
        return 1 / (1 + alpha) * (ind_mean + common_mean)

In [7]:
#Case: rho = -0.9, alpha = 1, b1 = 1, b2 = 1

#M is the length of the sample
M = 2000000
rho_gauss = [[1, -0.9], [-0.9, 1]]
alpha, b1, b2, c = 1., 1., 1., 1.
T = 1
lam = [2, 2]
lam_exp = [3, 3]
obj = CompoundPoisson2(T, lam,  M, lam_exp, rho_gauss)
rho_pois = obj.rho_pois()
distr = obj.get_compound_distr()
loss = MCLossFunction2(distr, alpha, b1, b2, c)

maxiter = 3500
init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init,jac = loss.objective_jac, constraints = cons, method='SLSQP',options={'maxiter': maxiter, 'disp': True})
print(res)
print(rho_pois)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4150951417458444
            Iterations: 5
            Function evaluations: 5
            Gradient evaluations: 5
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4150951417458444
            Iterations: 5
            Function evaluations: 5
            Gradient evaluations: 5
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4150951417458444
            Iterations: 5
            Function evaluations: 5
            Gradient evaluations: 5
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4150951417458444
            Iterations: 5
            Function evaluations: 5
            Gradient evaluations: 5
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4150951417458444
            Iterations: 5
            Function evaluations: 5
 

In [4]:
#Case: rho = -0.5, alpha = 1, b1 = 1, b2 = 1

#M is the length of the sample
M = 2000000
rho_gauss = [[1, -0.5], [-0.5, 1]]
alpha, b1, b2, c = 1., 1., 1., 1.
T = 1
lam = [2, 2]
lam_exp = [3, 3]
obj = CompoundPoisson2(T, lam,  M, lam_exp, rho_gauss)
rho_pois = obj.rho_pois()
distr = obj.compound_poisson2()
loss = MCLossFunction2(distr, alpha, b1, b2, c)

maxiter = 3500
init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, 
              jac = loss.objective_jac, 
              constraints = cons, 
               method='SLSQP',
               options={'maxiter': maxiter, 'disp': True})
print(res)
print(rho_pois)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4759720365681979
            Iterations: 5
            Function evaluations: 5
            Gradient evaluations: 5
     fun: 1.4759720365681979
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 5
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([0.73830375, 0.73766829])
[[ 1.         -0.45076225]
 [-0.45076225  1.        ]]


In [5]:
#Case: rho = -0.2, alpha = 1, b1 = 1, b2 = 1

#M is the length of the sample
M = 2000000
rho_gauss = [[1, -0.2], [-0.2, 1]]
alpha, b1, b2, c = 1., 1., 1., 1.
T = 1
lam = [2, 2]
lam_exp = [3, 3]
obj = CompoundPoisson2(T, lam,  M, lam_exp, rho_gauss)
rho_pois = obj.rho_pois()
distr = obj.compound_poisson2()
loss = MCLossFunction2(distr, alpha, b1, b2, c)

maxiter = 3500
init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, 
              jac = loss.objective_jac, 
              constraints = cons, 
               method='SLSQP',
               options={'maxiter': maxiter, 'disp': True})
print(res)
print(rho_pois)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5281696848628652
            Iterations: 5
            Function evaluations: 5
            Gradient evaluations: 5
     fun: 1.5281696848628652
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 5
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([0.76551428, 0.7626554 ])
[[ 1.         -0.18238146]
 [-0.18238146  1.        ]]


In [6]:
#Case: rho = 0, alpha = 1, b1 = 1, b2 = 1

#M is the length of the sample
M = 2000000
rho_gauss = [[1, 0], [0, 1]]
alpha, b1, b2, c = 1., 1., 1., 1.
T = 1
lam = [2, 2]
lam_exp = [3, 3]
obj = CompoundPoisson2(T, lam,  M, lam_exp, rho_gauss)
rho_pois = obj.rho_pois()
distr = obj.compound_poisson2()
loss = MCLossFunction2(distr, alpha, b1, b2, c)

maxiter = 3500
init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, 
              jac = loss.objective_jac, 
              constraints = cons, 
               method='SLSQP',
               options={'maxiter': maxiter, 'disp': True})
print(res)
print(rho_pois)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5798378852827684
            Iterations: 5
            Function evaluations: 5
            Gradient evaluations: 5
     fun: 1.5798378852827684
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 5
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([0.7900076 , 0.78983028])
[[1.00000000e+00 2.87955222e-04]
 [2.87955222e-04 1.00000000e+00]]


In [7]:
#Case: rho = 0.2, alpha = 1, b1 = 1, b2 = 1

#M is the length of the sample
M = 2000000
rho_gauss = [[1, 0.2], [0.2, 1]]
alpha, b1, b2, c = 1., 1., 1., 1.
T = 1
lam = [2, 2]
lam_exp = [3, 3]
obj = CompoundPoisson2(T, lam,  M, lam_exp, rho_gauss)
rho_pois = obj.rho_pois()
distr = obj.compound_poisson2()
loss = MCLossFunction2(distr, alpha, b1, b2, c)

maxiter = 3500
init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, 
              jac = loss.objective_jac, 
              constraints = cons, 
               method='SLSQP',
               options={'maxiter': maxiter, 'disp': True})
print(res)
print(rho_pois)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.619338730586866
            Iterations: 4
            Function evaluations: 5
            Gradient evaluations: 4
     fun: 1.619338730586866
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 5
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([0.80934992, 0.80998881])
[[1.         0.18454612]
 [0.18454612 1.        ]]


In [8]:
#Case: rho = 0.5, alpha = 1, b1 = 1, b2 = 1

#M is the length of the sample
M = 2000000
rho_gauss = [[1, 0.5], [0.5, 1]]
alpha, b1, b2, c = 1., 1., 1., 1.
T = 1
lam = [2, 2]
lam_exp = [3, 3]
obj = CompoundPoisson2(T, lam,  M, lam_exp, rho_gauss)
rho_pois = obj.rho_pois()
distr = obj.compound_poisson2()
loss = MCLossFunction2(distr, alpha, b1, b2, c)

maxiter = 3500
init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, 
              jac = loss.objective_jac, 
              constraints = cons, 
               method='SLSQP',
               options={'maxiter': maxiter, 'disp': True})
print(res)
print(rho_pois)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7034227508249251
            Iterations: 4
            Function evaluations: 4
            Gradient evaluations: 4
     fun: 1.7034227508249251
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 4
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([0.85189357, 0.85152918])
[[1.        0.4704811]
 [0.4704811 1.       ]]


In [9]:
#Case: rho = 0.9, alpha = 1, b1 = 1, b2 = 1

#M is the length of the sample
M = 2000000
rho_gauss = [[1, 0.9], [0.9, 1]]
alpha, b1, b2, c = 1., 1., 1., 1.
T = 1
lam = [2, 2]
lam_exp = [3, 3]
obj = CompoundPoisson2(T, lam,  M, lam_exp, rho_gauss)
rho_pois = obj.rho_pois()
distr = obj.compound_poisson2()
loss = MCLossFunction2(distr, alpha, b1, b2, c)

maxiter = 3500
init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, 
              jac = loss.objective_jac, 
              constraints = cons, 
               method='SLSQP',
               options={'maxiter': maxiter, 'disp': True})
print(res)
print(rho_pois)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8222167032854863
            Iterations: 4
            Function evaluations: 4
            Gradient evaluations: 4
     fun: 1.8222167032854863
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 4
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([0.91084126, 0.91137544])
[[1.         0.86049268]
 [0.86049268 1.        ]]


In [8]:
#Case: rho = -0.9, alpha = 0, b1 = 1, b2 = 1

#M is the length of the sample
M = 2000000
rho_gauss = [[1, -0.9], [-0.9, 1]]
alpha, b1, b2, c = 0, 1., 1., 1.
T = 1
lam = [2, 2]
lam_exp = [3, 3]
obj = CompoundPoisson2(T, lam,  M, lam_exp, rho_gauss)
rho_pois = obj.rho_pois()
distr = obj.compound_poisson2()
loss = MCLossFunction2(distr, alpha, b1, b2, c)

maxiter = 3500
init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
%timeit res = minimize(loss.objective, init, jac = loss.objective_jac, constraints = cons, method='SLSQP',options={'maxiter': maxiter, 'disp': True})
print(rho_pois)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9992468876641358
            Iterations: 2
            Function evaluations: 2
            Gradient evaluations: 2
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9992468876641358
            Iterations: 2
            Function evaluations: 2
            Gradient evaluations: 2
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9992468876641358
            Iterations: 2
            Function evaluations: 2
            Gradient evaluations: 2
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9992468876641358
            Iterations: 2
            Function evaluations: 2
            Gradient evaluations: 2
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9992468876641358
            Iterations: 2
            Function evaluations: 2
 

In [11]:
#Case: rho = -0.5, alpha = 0, b1 = 1, b2 = 1

#M is the length of the sample
M = 2000000
rho_gauss = [[1, -0.5], [-0.5, 1]]
alpha, b1, b2, c = 0, 1., 1., 1.
T = 1
lam = [2, 2]
lam_exp = [3, 3]
obj = CompoundPoisson2(T, lam,  M, lam_exp, rho_gauss)
rho_pois = obj.rho_pois()
distr = obj.compound_poisson2()
loss = MCLossFunction2(distr, alpha, b1, b2, c)

maxiter = 3500
init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, 
              jac = loss.objective_jac, 
              constraints = cons, 
               method='SLSQP',
               options={'maxiter': maxiter, 'disp': True})
print(res)
print(rho_pois)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9969803987578458
            Iterations: 3
            Function evaluations: 3
            Gradient evaluations: 3
     fun: 1.9969803987578458
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 3
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([0.99868857, 0.99829183])
[[ 1.         -0.45109185]
 [-0.45109185  1.        ]]


In [12]:
#Case: rho = 0, alpha = 0, b1 = 1, b2 = 1

#M is the length of the sample
M = 2000000
rho_gauss = [[1, 0], [0, 1]]
alpha, b1, b2, c = 0, 1., 1., 1.
T = 1
lam = [2, 2]
lam_exp = [3, 3]
obj = CompoundPoisson2(T, lam,  M, lam_exp, rho_gauss)
rho_pois = obj.rho_pois()
distr = obj.compound_poisson2()
loss = MCLossFunction2(distr, alpha, b1, b2, c)

maxiter = 3500
init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, 
              jac = loss.objective_jac, 
              constraints = cons, 
               method='SLSQP',
               options={'maxiter': maxiter, 'disp': True})
print(res)
print(rho_pois)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.001169235879098
            Iterations: 2
            Function evaluations: 3
            Gradient evaluations: 2
     fun: 2.001169235879098
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 3
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([1.00120656, 0.99996268])
[[1.00000000e+00 4.32480478e-04]
 [4.32480478e-04 1.00000000e+00]]


In [13]:
#Case: rho = 0.5, alpha = 0, b1 = 1, b2 = 1

#M is the length of the sample
M = 2000000
rho_gauss = [[1, 0.5], [0.5, 1]]
alpha, b1, b2, c = 0, 1., 1., 1.
T = 1
lam = [2, 2]
lam_exp = [3, 3]
obj = CompoundPoisson2(T, lam,  M, lam_exp, rho_gauss)
rho_pois = obj.rho_pois()
distr = obj.compound_poisson2()
loss = MCLossFunction2(distr, alpha, b1, b2, c)

maxiter = 3500
init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, 
              jac = loss.objective_jac, 
              constraints = cons, 
               method='SLSQP',
               options={'maxiter': maxiter, 'disp': True})
print(res)
print(rho_pois)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9961717904560565
            Iterations: 3
            Function evaluations: 3
            Gradient evaluations: 3
     fun: 1.9961717904560565
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 3
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([0.99892493, 0.99724686])
[[1.         0.46983166]
 [0.46983166 1.        ]]


In [14]:
#Case: rho = 0.9, alpha = 0, b1 = 1, b2 = 1

#M is the length of the sample
M = 2000000
rho_gauss = [[1, 0.9], [0.9, 1]]
alpha, b1, b2, c = 0, 1., 1., 1.
T = 1
lam = [2, 2]
lam_exp = [3, 3]
obj = CompoundPoisson2(T, lam,  M, lam_exp, rho_gauss)
rho_pois = obj.rho_pois()
distr = obj.compound_poisson2()
loss = MCLossFunction2(distr, alpha, b1, b2, c)

maxiter = 3500
init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, 
              jac = loss.objective_jac, 
              constraints = cons, 
               method='SLSQP',
               options={'maxiter': maxiter, 'disp': True})
print(res)
print(rho_pois)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.003155176110117
            Iterations: 3
            Function evaluations: 3
            Gradient evaluations: 3
     fun: 2.003155176110117
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 3
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([1.00203881, 1.00111636])
[[1.         0.86045134]
 [0.86045134 1.        ]]
